<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

In [ ]:
from IPython.display import display, HTML

In [ ]:
display(HTML("<style>.container { width:100% !important; }</style>"))

This notebook records the history of how I built this library from scratch. Well, not exactly from the scratch, because there is a [first version](https://github.com/EmbraceLife/debuggable) of this library where many difficulties have been already handled.

If you want to see the latest functions of this module, just `cmd/ctrl + f` to search '#| export' and check them out this way.

## make life easier with defaults  

In [ ]:
defaults = type('defaults', (object,), {'margin': 157, # align to the right by 157
                                        'orisrc': None, # keep a copy of original official src code
                                        'outenv': globals(), # outside global env
                                        'cmts': {} # a dict to store idx and cmt
                                        # 'eg': None, # examples
                                        # 'src': None, # official src
                                       })

### when exec update existing functions

In [ ]:
def foo(x, y): return x + y
def test(func):
    a = 1
    b = inspect.getsource(func)
    newb = ""
    for l in b.split('\n'):
        if bool(l):
            newb = newb + l
    newb = newb + " + 3\n"
    pprint(f'locals: {locals()}', width=157) # foo is not available in locals()
    print(f'exec(newb): {exec(newb)}') 
    pprint(f'locals: {locals()}', width=157) # a foo is available in locals(), but which one is it
    newfoo = locals()["foo"] # to use it, must assign to a different name
    print(newfoo(1,9))
    print(locals()["foo"](1,9))
    print(func(1,9))
    print(foo(1,9))

test(foo)

"locals: {'func': <function foo at 0x10f99df70>, 'a': 1, 'b': 'def foo(x, y): return x + y\\n', 'newb': 'def foo(x, y): return x + y + 3\\n', 'l': ''}"
exec(newb): None
("locals: {'func': <function foo at 0x10f99df70>, 'a': 1, 'b': 'def foo(x, y): return x + y\\n', 'newb': 'def foo(x, y): return x + y + 3\\n', 'l': '', "
 "'foo': <function foo at 0x10f9a0ee0>}")
13
13
10
10


### when the func to be udpated involve other libraries

In [ ]:
import functools

In [ ]:
def foo(x, y): 
    print(inspect.signature(foo))
    return x + y
def test(func):
    a = 1
    b = inspect.getsource(func)
    newb = ""
    for l in b.split('\n'):
        if bool(l) and "return" not in l :
            newb = newb + l + '\n'
        elif bool(l):
            newb = newb + l
    newb = newb + " + 3\n"
    pprint(f'locals: {locals()}', width=157) # foo is not available in locals()
    print(f'exec(newb): {exec(newb)}') 
    pprint(f'locals: {locals()}', width=157) # a foo is available in locals(), but which one is it
    newfoo = locals()["foo"]
    print(newfoo(1,9))
    print(locals()["foo"](1,9))
    print(func(1,9))
    print(foo(1,9))

test(foo)

("locals: {'func': <function foo at 0x10f99dd30>, 'a': 1, 'b': 'def foo(x, y): \\n    print(inspect.signature(foo))\\n    return x + y\\n', 'newb': 'def "
 "foo(x, y): \\n    print(inspect.signature(foo))\\n    return x + y + 3\\n', 'l': ''}")
exec(newb): None
("locals: {'func': <function foo at 0x10f99dd30>, 'a': 1, 'b': 'def foo(x, y): \\n    print(inspect.signature(foo))\\n    return x + y\\n', 'newb': 'def "
 "foo(x, y): \\n    print(inspect.signature(foo))\\n    return x + y + 3\\n', 'l': '', 'foo': <function foo at 0x10f9ac040>}")
(x, y)
13
(x, y)
13
(x, y)
10
(x, y)
10


### inside a function, exec() allow won't give you necessary env from function namespace

In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)
    exec(b) # create the new add in locals
    pprint(f'locals: {locals()}', width=157)
    print(f'add(5,6): {add(5,6)}')
    add1 = locals()['add'] # assign a different name, add1
    
    print(f'add1(5,6): {add1(5,6)}') # error: lst is not defined, even though lst=[] is right above
    
    pprint(f'locals: {locals()}', width=157)

try:
    test(add)
except NameError as e:
    print(e)

"locals: {'func': <function add at 0x10f99d940>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
("locals: {'func': <function add at 0x10f99d940>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 '0x106816550>}')
add(5,6): 1
name 'lst' is not defined


### magic of `exec(b, globals().update(locals()))`

What about `exec(b, globals().update(globals()))`

In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)
    
    exec(b, globals().update(globals())) # update(globals()) won't give us lst above
    
    pprint(f'locals: {locals()}', width=157)
    add1 = locals()['add'] 
    print(add1(5,6))
    pprint(f'locals: {locals()}', width=157)



try:
    test(add)
except: 
    print("exec(b, globals().update(globals())) won't give us lst in the func namespace")

"locals: {'func': <function add at 0x10f9a0dc0>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
("locals: {'func': <function add at 0x10f9a0dc0>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 '0x10f99ddc0>}')
exec(b, globals().update(globals())) won't give us lst in the func namespace


In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)
    
    exec(b, globals().update(locals())) # make sure b can access lst from above
    
    pprint(f'locals: {locals()}', width=157)
    add1 = locals()['add'] 
    print(add1(5,6))
    pprint(f'locals: {locals()}', width=157)

test(add)
print(add(5,6))
try:
    print(add1(5,6))
except: 
    print("you can't bring add1 from a function namespace to the outside world")

"locals: {'func': <function add at 0x10f99ddc0>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
("locals: {'func': <function add at 0x10f99ddc0>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 '0x10f99daf0>}')
11
("locals: {'func': <function add at 0x10f99ddc0>, 'a': 1, 'lst': [5], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add at "
 "0x10f99daf0>, 'add1': <function add at 0x10f99daf0>}")
1
you can't bring add1 from a function namespace to the outside world


### Bring variables from a func namespace to the sideout world

In [ ]:
def add(x, y): return 1

def test(func):
    a = 1
    lst = []
    b = "def add(x, y):\n    lst.append(x)\n    return x + y"
    pprint(f'locals: {locals()}', width=157)    
    
    exec(b, globals().update(locals())) # make sure b can access lst from above

    add1 = locals()['add'] 
    print(add1(5,6))
    add1(5,6)
    pprint(f'locals: {locals()}', width=157)

    # bring variables inside a func to the outside
    globals().update(locals())

test(add)
pprint(add(5,6)) # the original add is override by the add from the function's locals()
pprint(add1(5,6))
print(lst)

"locals: {'func': <function add at 0x10f99daf0>, 'a': 1, 'lst': [], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y'}"
11
("locals: {'func': <function add at 0x10f99daf0>, 'a': 1, 'lst': [5, 5], 'b': 'def add(x, y):\\n    lst.append(x)\\n    return x + y', 'add': <function add "
 "at 0x10f9ac310>, 'add1': <function add at 0x10f9ac310>}")
11
11
[5, 5, 5, 5]


### globals() in a cell vs globals() in a func

In [ ]:
from fastdebug.utils import tstenv

In [ ]:
len(globals().keys())

91

In [ ]:
globals()['__name__']

'__main__'

In [ ]:
tstenv??

In [ ]:
tstenv()

out global env has 20 vars
inner global env has 20 vars
inner local env has 20 vars
['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', '__all__', 'os']
['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__file__', '__cached__', '__builtins__', '__all__', 'os']
out env['__name__']: fastdebug.utils
inner env['__name__']: fastdebug.utils


## make a colorful string

In [1]:
#|output: asis
#| echo: false
show_doc(dbcolors)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core0.py#L24){target="_blank" style="float:right; font-size:smaller"}

### dbcolors

>      dbcolors ()

Initialize self.  See help(type(self)) for accurate signature.

In [2]:
#|output: asis
#| echo: false
show_doc(colorize)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core0.py#L31){target="_blank" style="float:right; font-size:smaller"}

### colorize

>      colorize (cmt, color:str=None)

## align text to the most right

In [3]:
#|output: asis
#| echo: false
show_doc(strip_ansi)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core0.py#L45){target="_blank" style="float:right; font-size:smaller"}

### strip_ansi

>      strip_ansi (source)

In [4]:
#|output: asis
#| echo: false
show_doc(alignright)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core0.py#L49){target="_blank" style="float:right; font-size:smaller"}

### alignright

>      alignright (blocks, margin:int=157)

In [5]:
#|output: asis
#| echo: false
show_doc(printsrclinewithidx)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core0.py#L60){target="_blank" style="float:right; font-size:smaller"}

### printsrclinewithidx

>      printsrclinewithidx (idx, l, fill=' ')

In [6]:
#|output: asis
#| echo: false
show_doc(printsrc)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core0.py#L67){target="_blank" style="float:right; font-size:smaller"}

### printsrc

>      printsrc (src, dbcode, cmt, expand:int=2)

print the seleted srcline with comment, idx and specified num of expanding srclines

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| src |  |  | name of src code such as foo, or delegates |
| dbcode |  |  | string of codes or int of code idx number |
| cmt |  |  |  |
| expand | int | 2 | expand the codes around the srcline under investigation |

In [7]:
#|output: asis
#| echo: false
show_doc(dbprintinsert)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core0.py#L108){target="_blank" style="float:right; font-size:smaller"}

### dbprintinsert

>      dbprintinsert (*codes, env={})

## Make fastdebug a class

### using @patch to enable docs for instance methods like `dbprint` and `print`

In [ ]:
class Fastdb():
    "Create a Fastdebug class which has two functionalities: dbprint and print."
    def __init__(self, 
                 src, # name of src code you are exploring
                 env): # env variables needed for exploring the source code, e.g., g = globals()
        self.orisrc = src
        self.margin = 157
        self.outenv = env
        self.cmts = {}

In [8]:
#|output: asis
#| echo: false
show_doc(Fastdb)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core0.py#L239){target="_blank" style="float:right; font-size:smaller"}

### Fastdb

>      Fastdb (src, env)

Create a Fastdebug class which has two functionalities: dbprint and print.

|    | **Details** |
| -- | ----------- |
| src | name of src code you are exploring |
| env | env variables needed for exploring the source code, e.g., g = globals() |

In [ ]:
@patch
def dbprint(self:Fastdb, 
            dbcode:int, # idx of a srcline under investigation, can only be int
            cmt:str, # comment added to the srcline
            *codes, # a list of expressions (str) you write to be evaluated above the srcline
            expand:int=2, # span 2 lines of srcode up and down from the srcline investigated
            showdbsrc:bool=False): # display dbsrc
    "Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code \
you are investigating. Run exec on the entire srcode with added expressions (dbsrc), so that dbsrc is callable."

    src = self.orisrc
    if type(dbcode) == int: self.cmts.update({dbcode: cmt})

    printsrc(src, dbcode, cmt, expand)
    print('{:-<60}'.format(colorize("print selected srcline with expands above", color="y")))
    
    dbsrc = ""
    indent = 4
    onedbprint = False

    lst = inspect.getsource(src).split('\n')
    if not bool(lst[-1]): lst = lst[:-1]

    newlst = []
    for i in codes: # no matter whether there is "" or "  " in the front or in the middle of codes
        if bool(i.strip()): newlst.append(i)
    codes = newlst

    srclines = ""
    if type(dbcode) == int:
        srclines = lst[dbcode]
    else:
        colwarn = colorize("Warning!", color="r")
        colmsg = colorize(" param decode has to be an int as idx.", color="y")
        print(colwarn + colmsg)
#         srclines = dbcode
        return

    for idx, l in zip(range(len(lst)), lst):

        if bool(l.strip()) and l.strip() in srclines and idx == dbcode:

            if len(codes) > 0: 
                numindent = len(l) - len(l.lstrip()) # make sure indent not messed up by trailing spaces
                dbcodes = "dbprintinsert("
                count = 1
                for c in codes:
                    if count == len(codes):
                        dbcodes = dbcodes + '"' + c + '"' + "," + "env=g" + ")"
                    else:
                        dbcodes = dbcodes + '"' + c + '"' + ","
                    count = count + 1

                dbsrc = dbsrc + " "*numindent + "g = locals()" + '\n'
                dbsrc = dbsrc + " "*numindent + dbcodes + '\n'
                dbsrc = dbsrc + l + '\n'     
            else:
                dbsrc = dbsrc + l + '\n'                

        elif bool(l.strip()) and idx + 1 == len(lst):
            dbsrc = dbsrc + l

        elif bool(l.strip()): # make sure pure indentation + \n is ignored
            dbsrc = dbsrc + l + '\n'

    if showdbsrc: # added to debug
        print('{:-<60}'.format(colorize("showdbsrc=Start", color="y")))
        totallen = 157
        lenidx = 5
        dblst = dbsrc.split('\n')
        for idx, l in zip(range(len(dblst)), dblst):
            lenl = len(l)
#             if "dbprintinsert" in l: 
            if l.strip().startswith("dbprintinsert"): 
                print(l + "="*(totallen-lenl-lenidx) + "(db)")
            else:
                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                
        names = self.orisrc.__qualname__.split('.')
        clsname = names[0]
        methodname = names[1]
        print(f"before exec, is {methodname} in locals(): {methodname in locals()}")
        print(f"before exec, is {clsname} in locals(): {clsname in locals()}")
        print(f"before exec, is {self.orisrc.__qualname__} in locals(): {self.orisrc.__qualname__ in locals()}")
        print(f"before exec, is {methodname} in self.outenv: {methodname in self.outenv}")
        print(f"before exec, is {clsname} in self.outenv: {clsname in self.outenv}")
        print(f"before exec, is {self.orisrc.__qualname__} in self.outenv: {self.orisrc.__qualname__ in self.outenv}")
        expr = "self.outenv[" + "'" + clsname + "']." + methodname
        expr1 = "self.outenv[" + "'" + methodname + "']"
        print(f"inspect.getsourcefile({expr}) == '<string>': {True if inspect.getsourcefile(eval(expr)) == '<string>' else inspect.getsourcefile(eval(expr))}")
        print(f"self.outenv[{methodname}]: {eval(expr1)}")
    exec(dbsrc, globals().update(self.outenv)) # make sure b can access lst from above
    print('{:-<60}'.format(colorize("exec on dbsrc above", color="y")))
    
    if showdbsrc: 
        print(f"locals() keys: {list(locals().keys())}")
        print(f"after exec, is {methodname} in locals(): {methodname in locals()}")
        print(f"after exec, is {clsname} in locals(): {clsname in locals()}")
        print(f"after exec, is {self.orisrc.__qualname__} in locals(): {self.orisrc.__qualname__ in locals()}")
        print(f"after exec, is {methodname} in self.outenv: {methodname in self.outenv}")
        print(f"after exec, is {clsname} in self.outenv: {clsname in self.outenv}")
        print(f"after exec, is {self.orisrc.__qualname__} in self.outenv: {self.orisrc.__qualname__ in self.outenv}")
#         print(f"after exec, are {methodname} and {clsname} and {self.orisrc.__qualname__} in locals(): {[i in list(locals().keys()) for i in [self.orisrc.__name__, clsname, self.orisrc.__qualname__]]}")
#         print(f"after exec, are {methodname} and {clsname} and {self.orisrc.__qualname__} in self.outenv(): {[i in self.outenv for i in [methodname, clsname, self.orisrc.__qualname__]]}")
        print(f"inspect.getsourcefile({expr}) == '<string>': {True if inspect.getsourcefile(eval(expr)) == '<string>' else inspect.getsourcefile(eval(expr))}")
        print(f"self.outenv[{methodname}]: {eval(expr1)}")
        print(f'self.orisrc.__name__: {self.orisrc.__name__}')
        print(f'locals()[self.orisrc.__name__]: {locals()[self.orisrc.__name__]}')
        print('{:-<60}'.format(colorize("showdbsrc=End", color="y")))
        
    return locals()[self.orisrc.__name__]

In [9]:
#|output: asis
#| echo: false
show_doc(Fastdb.dbprint)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core0.py#L251){target="_blank" style="float:right; font-size:smaller"}

### Fastdb.dbprint

>      Fastdb.dbprint (dbcode:int, cmt:str, *codes, expand:int=2,
>                      showdbsrc:bool=False)

Add comment and evaluate custom (single or multi lines) expressions to any srcline of the source code you are investigating. Run exec on the entire srcode with added expressions (dbsrc), so that dbsrc is callable.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| dbcode | int |  | idx of a srcline under investigation, can only be int |
| cmt | str |  | comment added to the srcline |
| codes |  |  |  |
| expand | int | 2 |  |
| showdbsrc | bool | False |  |

In [ ]:
@patch
def print(self:Fastdb, 
            maxlines:int=33, # maximum num of lines per page
            part:int=0): # if the src is more than 33 lines, then divide the src by 33 into a few parts
    "Print the source code in whole or parts with idx and comments you added with dbprint along the way."

    totallen = 157
    lenidx = 5
    lspace = 10
    lstsrc = inspect.getsource(self.orisrc).split('\n')
    numparts = len(lstsrc) // 33 + 1 if len(lstsrc) % 33 != 0 else len(lstsrc) // 33
    cmts = self.cmts
    if part == 0: 
        for idx, l in zip(range(len(lstsrc)), lstsrc):
            lenl = len(l)

            if not bool(l.strip()):
                print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")

            elif lenl + lspace >= 100:
                if bool(cmts):
                    cmtidx = [cmt[0] for cmt in list(cmts.items())]
                    if idx in cmtidx:
                        print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                    else:
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                else: 
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")

            else:


                if bool(cmts):
                    cmtidx = [cmt[0] for cmt in list(cmts.items())]
                    if idx in cmtidx:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                    else:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                      

                else:
                    print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                 

    for p in range(numparts):
        for idx, l in zip(range(len(lstsrc)), lstsrc):

            if (maxlines*p <= idx < maxlines*(p+1) and p+1 == part):
                lenl = len(l)
                if not bool(l.strip()):
                    print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                elif lenl + lspace >= 100:
                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print(l + " # " + cmts[idx] + " "*(totallen-lenl-lenidx-len(cmts[idx])-3) + "(" + str(idx) + ")")
                        else:
                            print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")
                    else: 
                        print(l + " "*(totallen-lenl-lenidx) + "(" + str(idx) + ")")


                else:

                    if bool(cmts):
                        cmtidx = [cmt[0] for cmt in list(cmts.items())]
                        if idx in cmtidx:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})" + " # " + cmts[idx]))
                        else:
                            print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                                                          

                    else:
                        print('{:<100}'.format(l + "="*(100-lenl-lspace) + f"({idx})"))                      

            if (idx == maxlines*(p+1) or idx == len(lstsrc) - 1) and p+1 == part:
                print('{:>157}'.format(f"part No.{p+1} out of {numparts} parts"))
                return

In [10]:
#|output: asis
#| echo: false
show_doc(Fastdb.print)

---

[source](https://github.com/EmbraceLife/fastdebug/blob/master/fastdebug/core0.py#L365){target="_blank" style="float:right; font-size:smaller"}

### Fastdb.print

>      Fastdb.print (maxlines:int=33, part:int=0)

Print the source code in whole or parts with idx and comments you added with dbprint along the way.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| maxlines | int | 33 | maximum num of lines per page |
| part | int | 0 | if the src is more than 33 lines, then divide the src by 33 into a few parts |

### use dbprint to override the original official code without changing its own pyfile

see the example [here](./examples/dbprint.ipynb#make-inspect.signature-to-run-our-dbsrc-code)

```python
dbsig = sig.dbprint(29, "why has to unwrap?", "hasattr(obj, '__signature__')")
inspect._signature_from_callable = dbsig
pprint(inspect.signature(Foo))
sig.print(part=1)
```

### use guide on Fastdb.dbprint

1. don't use for the line start with `elif`, as putting [`dbprintinsert`](https://EmbraceLife.github.io/fastdebug/core-copy1.html#dbprintinsert) above `elif` without indentation will cause syntax error. I am not sure whether I need to fix it now.

see example [here](./examples/Fastdb.ipynb)

test it with example [here](./examples/print.ipynb)